# Development 

## Image to Value(s)

### Primary Focus: Explainable Object Counting in Microscopy Images
### Application: Explainable Virus Capsid Quantification
#### Challenge: Deep Learning as Black Box
#### Required Labels: Location Labels


TL;DR 🧬✨ We developed a regression model to quantify virus capsids and their mutation stages ("naked" 🩲, "budding" 🌱, "enveloped" 📦) during secondary envelopment in TEM images. Researchers can adapt the provided notebook within the primary focus area for their own EM data analysis (i.e. counting midrochondia).

![Teaser](./images/Teaser.png)




# Setup and Imports

In [1]:
# auto reload imports
%load_ext autoreload
%autoreload 2

# imports from the template 
from deepEM.Utils import create_text_widget, print_info, find_file
from deepEM.Logger import Logger
from deepEM.ModelTuner import ModelTuner

# costum implementation
from src.ModelTrainer import ModelTrainer


# import all required libraries
from pathlib import Path 
import os



# 1. Data

## 1.1. Data Acquisition
For exemplary purposes, we utilize existing data from [1]. However, we strongly encourage researchers to provide their own datasets tailored to their laboratory's specific needs. This approach enables training models optimized for the lab's unique sample preparation techniques and microscope attributes, such as detector configurations. Providing a different dataset can also address different types of application, like counting of different objects within an EM image.

----
*[1] Shaga Devan, Kavitha, et al. "Improved automatic detection of herpesvirus secondary envelopment stages in electron microscopy by augmenting training data with synthetic labelled images generated by a generative adversarial network." Cellular Microbiology 23.2 (2021): e13280.*

## 1.2. Data Anntation

This notebook requires annotations of object locations along with their corresponding classes.

The example dataset includes annotations for the locations of virus capsids within an image, categorized by their envelopment stages, $C=[’naked’,’budding’,’enveloped’]$.

To adapt the application of this notebook, EM researchers can provide their own dataset. 
In the following, we outline an exemplary workflow for generating annotation labels for the specific task of predicting the number of virus capsids and their corresponding envelopment stages.

For data annotation we recomment using the <a href="https://www.cvat.ai/">CVAT</a> (Computer Vision Annotation Tool) tool. For further instructions, we refer to our <a href="">Getting Started</a> page.



### 1.2.1. Create a New Task

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-1.png" alt="Create New Task1" width="500">
  <br>
</p>
When starting CVAT, you first need to create a new task. You can give it a name, add annotation types and upload your data.

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-2.png" alt="Create New Task2" width="500">
  <br>
</p>

Next, click on the `Add label` button. Name it based on the class you want to annotate. In our case one of *"naked", "budding", "enveloped"*. As annotation type choose `Points`. You should also pick a color, as this will simplify the annotation process. For adding new class click `Continue`. Once you added all nessecary classes click `Cancel`. 


<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-3.png" alt="Create New Task3" width="500">
  <br>
</p>

Now you can upload the data you wish to annotate. Finally, click `Submit & Open` to continue with the annotation of the uploaded data. 

### 1.2.2. Annotation

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-4.png" alt="Annotate1" width="500">
  <br>
</p>

This will open following view. Click on the job (in this view the `Job #1150022`) to start the annotation job. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-5.png" alt="Annotate2" width="500">
  <br>
</p>

To then annotate your data, select the `Draw new points` tool. Select the Label you wish to annotate from the dropdown menue. Then click `Shape` to annotate individual virus capsids with the label class. (Track will allow you to place annotations over multiple frames, which is helpful when annotating videos, tomograms or similar). You can use the arrows on the top middle to navigate through all of your data and to see your annotation progress. 

### 1.2.3. Save Annotation

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-6.png" alt="Save1" width="500">
  <br>
</p>

Once you are done annotating data, click on the `Menu` and select `Export job dataset`. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-7.png" alt="Save2" width="500">
  <br>
</p>

During export select the `CVAT for Images 1.1` format and give the folder a name. It will prepare the dataset for download. If you have the annotated images stored locally, there is no need to enable `Save Images`. 

<p align="center">
  <img src="https://viscom-ulm.github.io/DeepEM/static/images/explainable-virus-quantification/CVAT-8.png" alt="Save3" width="500">
  <br>
</p>

In the horizontal menu bar at the top go to `Requests`. It will show a request Export Annotations. On the right of this request click on the three dots on the right to download your exported, annotated data. This will download a .zip file containing the annotation file in .xml format. The name of the file should be "annotations.xml".



## 1.3. Data Preprocessing

The provided notebook requires all images to be in `.tif` format, containing single 2D images. If this is not the case for your data, you can use the [ImageJ](https://imagej.net/ij/) software. 
ImageJ is an open-source, Java-based image processing software that runs on multiple platforms and offers a wide range of features, including automation with macros, extensive community support, and a large library of tools and plugins.

In the following we showcase an example usecase to import and export data into required formats. 



<p align="center">
  <img src="./images/ImageJ-1.png" alt="ImageJ1" width="500">
  <br>
</p>

This tool allows to `import` a large amount of different, commonly used file formats im EM. 

<p align="center">
  <img src="./images/ImageJ-2.png" alt="ImageJ2" width="500">
  <br>
</p>

Using the provided `Save As..` functionality allows to save the imported files as a `Image Sequence` in .tif format or single `.tif` files.

## 1.4. Data Structuring

The provided notebook requires that all training, validation and testing data is placed within a single folder. Splitting the data into train, test and validation will be done during runtime. 

Additionally, the generated `annotations.xml` should be put in the same folder as the .tif images.

You can check the exemplary data provided at `data/tem-herpes/` for clarification.

An example with five images and the corresponding annotation is shown below: 

```
/data/tem-herpes/
├── image_001.tif
├── image_002.tif
├── image_003.tif
├── image_004.tif
├── image_005.tif
└── annotations.xml

```

If you are using different data than the one provided (`data/tem-herpes/`), we require you to set the path to this folder. To do so, adapt the path in the text form below.

*Hint: In order to make the text form show, you need to run the cell below first.*


In [2]:
data_widget = create_text_widget("Data Path:","./data/tem-herpes","Enter the path to your data folder.")
display(*data_widget)

Text(value='./data/tem-herpes', description='Data Path:', style=TextStyle(description_width='initial'))

HTML(value='<b>Hint:</b> Enter the path to your data folder.')

In [3]:
data_path = data_widget[0].value
print(f"[INFO]::Data path was set to: {data_path}")

[INFO]::Data path was set to: ./data/tem-herpes


As high resolution EM images can usually not be processed in full size by DL models, the image sizes need to be adapted. In this use case, images will be cropped to a suitable input resolution of the model (224px x 224px). For inference, the image patches will be stiched together to reconstruct the full image. 

# 2. Model Training

## 2.1. Setup Logging

By executing the cell below, we setup the logging directory for model training and evaluation. 
The logger creates a folder at `./logs/<datafoldername>-<currentdatetime>/`. 
Within this folder there will be logging of: 

- the used hyperparameters, (`<log-path>/TrainingRun/hyperparameters.json`)
- the best performing model checkpoint based on the validation loss (`<log-path>/TrainingRun/checkpoints/best_model.pth`)
- the last model checkpoint (`<log-path>/TrainingRun/checkpoints/latest_model.pth`)
- visualizations of training and validation curves (`<log-path>/TrainingRun/plots/training_curves.png`)
- qualitative visualization of sampled validation images (`<log-path>/TrainingRun/samples/`)
- results on test metrics (`<log-path>/TrainingRun/test_results.txt`)
- qualitative visualization of sampled test images (`<log-path>/TrainingRun/samples/`)

Additional text logs are being saved to `<log-path>/TrainingRun/log.txt` and `<log-path>/log.txt` 

Sample visualizations of this use case include the model input, validation labels, predictions, and a GradCAM overlay. GradCAM can give an intuition about "where the model looks" to make its prediction.


In [4]:
logger = Logger(data_path)

Logger initialized. Logs will be saved to: logs/tem-herpes_2025-03-24_10-44-48


## 2.2. Hyperparameter Tuning


Hyperparameters in deep learning are configurable settings that define how a model is trained. Unlike model parameters, they are set before training and not learned from the data.

When training a model, we highly recommend to do a hyperparameter tuning first. By tuning the hyperparameters the model is usually trained on a subset of the data with a smaller number of epochs, and then evaluated based on its performance on the validation set. Then, hyperparameters, which lead to the best performance are chosen for full training of the model. 
Similar to the training run, all sweep runs will be logged. You can find the according logs at `<log-path>/Sweep-<idx>`.

Our workflow equips you, as EM experts, with an automatic hyperparameter search based on a grid search. The DL experts have chosen some basic setting for performing the hyperparameter seach and defining the search space. The DL experts also describe the individual hyperparameters. This allows you to further adapt the search space to your specific needs. 

In order to do so, you can adapt the form below. Each sweep parameter should be separated by `,`. Floating point values should be written like `0.1`. 

In [5]:
# hyperparameter search
model_trainer = ModelTrainer(data_path, logger)

hyperparameter_tuner = ModelTuner(model_trainer, data_path, logger)
form = hyperparameter_tuner.create_hyperparameter_widgets()
display(form)


If you wish to run a hyperparameter sweep based on the parameters above, please execute the cell below.

In [ ]:
best_config = None
hyperparameter_tuner.update_config(form)
print("Sweep config:")
for k in hyperparameter_tuner.config['hyperparameter'].keys():
    print(f"\t{k}: {hyperparameter_tuner.config['hyperparameter'][k]['values']} (default: {hyperparameter_tuner.config['hyperparameter'][k]['default']})")
best_config = hyperparameter_tuner.tune()

Our automatic hyperparameter tuning is able to find the best performing set of hyperparameters based on the setting shown above. 

However, there can be scenarios, where additional flexibility is required. Therefore, you are able to change these hyperparameters in the following. 

**WARNING** This setting is for advanced users only. Please only change parameters here, if you know what you are doing. 

In [6]:
form = hyperparameter_tuner.edit_hyperparameters()
display(form)

In [7]:
best_config = hyperparameter_tuner.update_hyperparameters(form)
print_info(f"Will use following hyperparameters for future training: {best_config}")

[INFO]::Will use following hyperparameters for future training: {'learning_rate': 1e-05, 'batch_size': 16}


## 2.3. Training and Validation

Training in deep learning is the process where a model learns patterns from labeled data by optimizing its parameters through backpropagation. 
Validation involves using a separate dataset to evaluate the model's performance during training, ensuring it generalizes well to unseen data.
Hence, in this section we train and validate the model based on the provided data and hyperparameters resulting from the previous sweep.

If no sweep was conducted (not recommended for new datasets!), the default parameters, defined by the DL expert will be used. 

In case the training run was cancelled, it can be resumed from a previous checkpoint. To do so, you need to provide a model checkpoint in the text form below. You can find these checkpoints inside the runs logging directory (`<log-dir>/TrainingRun/checkpoints/latest_model.pth`). If you do not wish to resume training, you can ignore the text form below.

In [8]:
resume_widget = create_text_widget("Resume Training:","","If you wish to resume an earlier training, enter the path to the latest_model.pth file here.")
display(*resume_widget)

Text(value='', description='Resume Training:', style=TextStyle(description_width='initial'))

HTML(value='<b>Hint:</b> If you wish to resume an earlier training, enter the path to the latest_model.pth fil…

In [13]:
resume_training = resume_widget[0].value
if(resume_training):
    resume_training = Path(resume_training)
    if(resume_training.is_dir()):
        resume_training = Path(os.path.join(resume_training,"latest_model.pth"))
    if(not resume_training.is_file()):
        logger.log_error(f"Could not find resume path at {resume_training}. Will start training from scatch.")
        resume_training = None
else:
    resume_training = None
logger.init("TrainingRun")
model_trainer.resume_from_checkpoint = resume_training
model_trainer.prepare(best_config)


2025-03-24 10:48:16,226 - INFO - Hyperparameters saved to logs/tem-herpes_2025-03-24_10-44-48/TrainingRun/hyperparameters.json


Logger initialized. Logs will be saved to: logs/tem-herpes_2025-03-24_10-44-48/TrainingRun
Setup Dataset with 3 classes: ['naked', 'budding', 'enveloped'] and 212 micrographs.
Setup Dataset with 3 classes: ['naked', 'budding', 'enveloped'] and 71 micrographs.
Setup Dataset with 3 classes: ['naked', 'budding', 'enveloped'] and 71 micrographs.


If you wish to train a model, execute the cell below. 

In [14]:
model_trainer.fit()

2025-03-24 10:48:19,831 - INFO - Start Training | Epoch: 3000 | Dataset size: 212 | Parameters: {'epochs': 3000, 'early_stopping_patience': 100, 'validation_interval': 100, 'scheduler_step_by': 'epoch', 'images_to_visualize': 10, 'learning_rate': 1e-05, 'batch_size': 16} 


2025-03-24 10:48:22,524 - INFO - Epoch 0 - Training loss: 0.0175
2025-03-24 10:48:26,754 - INFO - Saved visualizations to logs/tem-herpes_2025-03-24_10-44-48/TrainingRun/samples/validation_epoch-0_*
2025-03-24 10:48:37,449 - INFO - Epoch 0 - Validation loss: 0.0460, MAE: 0.0320, FN: 0.0000, FP: 0.0960, TP: 0.0000, MAE-naked: 0.0191, FN-naked: 0.0000, FP-naked: 0.0191, TP-naked: 0.0000, MAE-budding: 0.0523, FN-budding: 0.0000, FP-budding: 0.0523, TP-budding: 0.0000, MAE-enveloped: 0.0247, FN-enveloped: 0.0000, FP-enveloped: 0.0247, TP-enveloped: 0.0000
2025-03-24 10:48:37,893 - INFO - Current model checkpoint saved to logs/tem-herpes_2025-03-24_10-44-48/TrainingRun/checkpoints/latest_model.pth
2025-03-24 10:48:38,478 - INFO - Best model checkpoint saved to logs/tem-herpes_2025-03-24_10-44-48/TrainingRun/checkpoints/best_model.pth (Validation Loss: 0.0460)
2025-03-24 10:48:38,959 - INFO - Saved best model checkpoint for inference to logs/tem-herpes_2025-03-24_10-44-48/TrainingRun/checkpo

KeyboardInterrupt: 

In [15]:
print("Class distribution during training")
model_trainer.train_loader.dataset.transform_crop.print_class_distribution()

Class distribution during training
Current class distribution:
Class: naked, Count: 9936
Class: budding, Count: 9935
Class: enveloped, Count: 9934


# 3. Model Evaluation
Evaluation in deep learning is the process of evaluating a trained model on a separate, unseen dataset to measure its final performance. It provides an unbiased assessment of the model's ability to generalize to new data.

## 3.1. Choose Model 

In this section we choose the model for testing. 
If you leave the `Model Path` empty in the text form below, it will use the last model trained.
Otherwise, you can define the path to the models best weights at `<log-path>/TrainingRun/checkpoints/best_model.pth` or by providing a path to a directory, which contains `best_model.pth` (like `<log-path>/TrainingRun/`)

In [ ]:
model_widget = create_text_widget("Model Path:","","If you wish to test a specific model, you can here define the path to its checkpoint. (For example: logs/tem-herpes_2025-02-03_11-42-43/TrainingRun/checkpoints)")
display(*model_widget)

## 3.2. Evaluate
We finally evaluate the provided model on the test set. We investigate following metrics: 

#### **False Positive (FP)**
- **Definition**: A **false positive** occurs when the model **detects an object that is not actually there**.  
- **Example**: Imagine an electron microscopy image where the model highlights a structure as a capsid, but in reality, it is just noise. This would be a **false positive** because the detection is incorrect.  
- **Explanation**: The **FP value** represents the **average number of incorrectly detected objects** per image patch—cases where the model mistakenly finds objects that do not exist.  

---

#### **False Negative (FN)**
- **Definition**: A **false negative** occurs when the model **fails to detect an actual object** that is present in the image.  
- **Example**: Suppose there is a capsid in the image, but the model **does not recognize it**. This is a **false negative** because an actual object was missed.  
- **Explanation**: The **FN value** represents the **average number of real objects that the model failed to detect** per image patch.  

---

#### **True Positive (TP)**
- **Definition**: A **true positive** occurs when the model correctly detects an object that is actually present.  
- **Example**: If there is a capsid in the image and the model correctly detects it, this is a **true positive**.  
- **Explanation**: The **TP value** represents the **average number of correctly detected objects** per image patch—cases where the model successfully identifies real objects.  

---

#### **Mean Absolute Error (MAE)**
- **Definition**: The **Mean Absolute Error (MAE)** is a metric that measures the **average absolute difference** between the predicted and actual number of objects per image patch.  
- **Explanation**:  
  - MAE **measures how far off** the model’s predictions are from the true values on average.  
  - A **lower MAE** indicates better performance, as it means the predicted object count is closer to the actual count.  
- **Example**:  
  - If an image patch contains **5 objects** but the model predicts **7**, the absolute error is **|5 - 7| = 2**.  
  - The MAE across all patches provides an overall error measure.  

---

#### **Summary**
| Metric  | Meaning | Interpretation |
|---------|---------|---------|
| **FP**  | Wrongly detected objects (model detects something that isn't there). | Lower is better |
| **FN**  | Missed objects (model fails to detect an actual object). | Lower is better | 
| **TP**  | Correctly detected objects (model successfully identifies real objects). | Higher is better |
| **MAE** | Average absolute difference between predicted and actual object counts. | Lower is better |

Each metric is computed as the average for each input patch. (For example TP=2 means that the model correctly predicts 2 virus capsides per input image patch.) Additionally, we give the metrics as sum of all classes and additionally report the metrics for each class individually.

We further visualize the input image, the model prediction and use GradCAM to highlight areas where the model was "looking" to make its predictions. 
These visualizations are saved to `<log-path>/TrainingRun/samples/test_*`. 

If you wish to evaluate a model, execute the cell below.

In [ ]:
from pathlib import Path 
start_evaluation = False
eval_model = model_widget[0].value
if(eval_model):
    eval_model = Path(eval_model)
    if(eval_model.is_dir()):
        eval_model = Path(find_file(eval_model, "best_model.pth")) 
    if(not eval_model.is_file()):
        logger.log_error(f"Could not find model at {eval_model}. Make sure to train a model before evaluation.")
        eval_model = None
    else: 
        start_evaluation = True
else:
    recent_logs = logger.get_most_recent_logs()
    eval_model = ""
    for dataname, log_path in recent_logs.items():
        if(dataname == Path(data_path).stem):
            eval_model = Path(log_path+"/TrainingRun/checkpoints/best_model.pth")
            if(not eval_model.is_file()):
                logger.log_error(f"Cound not find a trained model at {eval_model}. Make sure you fully train a model first before evaluating.")
            else:
                logger.log_info(f"Found most recent log at {eval_model}")
                start_evaluation = True
        else: 
            continue
    if(not start_evaluation):
        logger.log_error("Cound not find a trained model. Make sure you train a model first before evaluating.")
      
if(start_evaluation):
    model_trainer.load_checkpoint(eval_model)
    model_trainer.test()      

 

